In [1]:
%matplotlib inline

from __future__ import division, print_function

import glob as glob

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy import optimize

sns.set_style('white')

ModuleNotFoundError: No module named 'matplotlib'

### Set parameters

In [ ]:
N_samples = 10000
threshold = .5 
REWARD = 1

_results_index = [
    'High Effort Success', 'Low Effort Success', 'High Effort Failure', 
    'Low Effort Failure', 'Baseline'
]

human_means = [pd.Series([107, 57, 30, 35,86], index=_results_index),
               pd.Series([96, 87, 38, 39,59], index=_results_index),
               pd.Series([171, 94, 62, 45,81], index=_results_index)]

### Free parameters to fit

In [ ]:
#COST = 0.5
#B=6
#A=60 
#u_skill = .625 #mean of beta
#tao_skill = 20 #variance of beta
#skill=.4
#scale = 200


### Functions

In [ ]:
def prob_reward(e,diff,skill, A, B):
    """Return the probability of getting the reward
    
    Parameters
    ----------
    e : numeric or array-like
        Description of this parameter.
    # TODO: list other parameters
    
    Returns
    -------
    ...
    """
    return 1/(1 + A * np.exp(-(e * skill / diff) * B)) 


def get_effort_multidim(cost, diff, skill, A, B, reward=1):
    """Similar to `get_effort` but accepts arrays or single values of `diff` 
    and `skill`.

    Parameters
    ----------
    ...

    Returns
    -------
    Ndarray with same shape as `diff` and `skill`.
    """
    es = np.linspace(0, 1, 100)
    es_rank2 = es[np.newaxis, ...]
    
    try:
        diff = diff[..., np.newaxis]
    except TypeError:
        pass
    try:
        skill = skill[..., np.newaxis]
    except TypeError:
        pass
    
    prob = prob_reward(e=es_rank2, diff=diff, skill=skill, A=A, B=B)
    utilities = reward * prob - cost * es_rank2
    return es[utilities.argmax(-1)]


def your_eff_know_skill_opt(n_samples, cost, diff, skill, A, B,reward=1):
    """Model for planning.

    Parameters
    ----------
    n_samples : int
    cost : numeric
    diff : array-like
    skill : numeric
    A : numeric
    B : numeric
    reward : numeric
    
    Returns
    -------
    Ndarray of effort.
    """
    rand_diffs = np.random.choice(diff, size=n_samples)
    return get_effort_multidim(
        cost=cost, diff=rand_diffs, skill=skill, A=A, B=B, reward=reward
    )


def create_priors_opt(n_samples, tao_skill, u_skill, cost, A, B, diff, reward=1, 
                      effort_threshold=0.5):
    """Documentation."""
    # Skill and diff are beta distributions.
    skills = np.random.beta(
        tao_skill * u_skill, (1 - u_skill) * tao_skill, size=n_samples,
    )
    
    efforts = get_effort_multidim(
        cost=cost, skill=skills, diff=diffs, A=A, B=B,
    )

    high_effort = efforts > effort_threshold
    success = prob_reward(efforts, diffs, skills, A, B) > np.random.random(n_samples)

    return skills,  success, high_effort


def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

### Function that takes in free parameters and spits out model predictions

In [ ]:
def run_model_opt(cost, A, B, u_skill, tao_skill, skill, scale, u_diff = 0.5, tao_diff = 1, n_samples=10000):
    np.random.seed(0)
    REWARD=1
    high_diff = .8
    low_diff = .2
    
    high_diff_skills, high_diff_success, high_diff_high_effort = create_priors_opt(
        n_samples =n_samples, tao_skill=tao_skill, u_skill=u_skill, cost=cost, A=A, B=B, diff = high_diff
    )

    low_diff_skills, low_diff_success, low_diff_high_effort = create_priors_opt(
        n_samples =n_samples, tao_skill=tao_skill, u_skill=u_skill, cost=cost, A=A, B=B, diff = low_diff
    )
    

    skill_high_diff_high_eff = high_skills[high_success & high_diff_high_effort]
    skill_high_diff_low_eff = skills[success & high_diff & low_effort]
    skill_low_diff_high_eff = skills[success & low_diff & high_effort]
    skill_low_diff_low_eff = skills[success & low_diff & low_effort]
    
    
    effort_high_success = your_eff_know_skill_opt(n_samples, cost, diffs[success & high_effort],skill, A,B) 
    effort_low_success = your_eff_know_skill_opt(n_samples,  cost, diffs[success & (~high_effort)],skill,A,B) 
    effort_high_fail = your_eff_know_skill_opt(n_samples,  cost, diffs[(~success) & high_effort],skill,A,B) 
    effort_low_fail = your_eff_know_skill_opt(n_samples,  cost, diffs[(~success) & (~high_effort)],skill,A,B) 
    effort_baseline = your_eff_know_skill_opt(n_samples,  cost, np.random.beta(tao_diff*u_diff, (1-u_diff)*tao_diff, size=n_samples),skill,A,B)

    model_means = np.stack(
        (effort_high_success, effort_low_success, effort_high_fail, 
         effort_low_fail, effort_baseline),
    ).mean(-1)
    model_means *= scale
    
    return model_means

### Optminize function

In [ ]:
def wrapped_run_model_opt_1(x):
    model_means = run_model_opt(*x)
    loss = rmse(model_means, human_means[0])
    print (loss, x)
    return loss


# key_words=dict(cost = 0.5,B=6,A=60 ,u_skill = .625 ,tao_skill = 20, skill=.4, scale =200)
# rmse_val= run_model_opt(**key_words)

inputs = np.array([0.5, 60, 6, .625, 20, .4, 200])
minimum = optimize.fmin(wrapped_run_model_opt_1, inputs)
minimum

### Optimized parameters

In [ ]:
key_words=dict(cost =  0.40009476,B=6.71213053,A=55.69216106 ,u_skill =  0.70834897,
               tao_skill = 18.78977603, skill=0.41762985, scale =201.32613568)

In [ ]:
model_means= run_model_opt(human_means[0], **key_words)[1]
d = {
    'Model' : pd.Series(model_means, index=_results_index),
    'Human' : human_means[0]
}

df = pd.DataFrame(d)
df

## Experiment 2
* difficulty

In [ ]:
def wrapped_run_model_opt_2(x):
    rmse = run_model_opt(
                         cost=key_words['cost'], 
                         A=key_words['A'], 
                         B=key_words['B'], 
                         u_skill=key_words['u_skill'], 
                         tao_skill=key_words['tao_skill'], 
                         skill=key_words['skill'], 
                         scale=key_words['scale'],
                         u_diff = x[0],
#                          tao_diff = x[1]
                        )
#     print(rmse, x)
    loss = rmse(model_means, human_means[1])

    return loss


# key_words=dict(cost = 0.5,B=6,A=60 ,u_skill = .625 ,tao_skill = 20, skill=.4, scale =200)
# rmse_val= run_model_opt(**key_words)

minimum = optimize.fmin(wrapped_run_model_opt_2, [0.7])
minimum

In [ ]:
model_means= run_model_opt(human_means[1], 
                           u_diff=.5, 
                           tao_diff=1, 
                           **key_words)[1]
d = {
    'Model' : pd.Series(model_means, index=_results_index),
    'Human' : human_means[1]
}

df = pd.DataFrame(d)
df

## Experiment 3
* moral 

In [ ]:
def wrapped_run_model_opt_3(x):
    rmse = run_model_opt(
                         cost=x, 
                         A=key_words['A'], 
                         B=key_words['B'], 
                         u_skill=key_words['u_skill'], 
                         tao_skill=key_words['tao_skill'], 
                         skill=key_words['skill'], 
                         scale=key_words['scale']
                        )
    loss = rmse(model_means, human_means[2])

    print(loss, x)
    return loss


# key_words=dict(cost = 0.5,B=6,A=60 ,u_skill = .625 ,tao_skill = 20, skill=.4, scale =200)
# rmse_val= run_model_opt(**key_words)

minimum = optimize.fmin(wrapped_run_model_opt_3, .4)
minimum
words = dict(key_words)
words['cost'] = minimum[0]
model_means= run_model_opt(human_means[2], 
                           **words)[1]
d = {
    'Model' : pd.Series(model_means, index=_results_index),
    'Human' : human_means[2]
}

df = pd.DataFrame(d)
df

## Joint Optimization across 3 experiments

In [ ]:
def wrapped_run_model_opt_joint(x):
    exp1 = run_model_opt(
        cost = x[0], 
        A = x[1], 
        B = x[2], 
        u_skill = x[3], 
        tao_skill = x[4], 
        skill = x[5], 
        scale = x[6], 
        u_diff = 0.5, 
    )
    
    exp2 = run_model_opt(
        cost = x[0], 
        A = x[1], 
        B = x[2], 
        u_skill = x[3], 
        tao_skill = x[4], 
        skill = x[5], 
        scale = x[6], 
        u_diff = x[7], 
    )
    
    exp3 = run_model_opt(
        cost = x[8], 
        A = x[1], 
        B = x[2], 
        u_skill = x[3], 
        tao_skill = x[4], 
        skill = x[5], 
        scale = x[6], 
        u_diff = 0.5, 
    )        
    model = np.array(list(exp1)+list(exp2)+list(exp3))
    human = np.array(list(human_means[0])+list(human_means[1])+list(human_means[2]))
    loss = rmse(model, human)
    print(loss, x)

    return loss


# key_words=dict(cost = 0.5,B=6,A=60 ,u_skill = .625 ,tao_skill = 20, skill=.4, scale =200)
# rmse_val= run_model_opt(**key_words)

minimum = optimize.fmin(wrapped_run_model_opt_joint, [   0.40009476,   55.69216106,    6.71213053,    0.70834897,
         18.78977603,    0.41762985,  201.32613568, .6, .3])
minimum

In [ ]:
    x = minimum
    exp1 = run_model_opt(
        cost = x[0], 
        A = x[1], 
        B = x[2], 
        u_skill = x[3], 
        tao_skill = x[4], 
        skill = x[5], 
        scale = x[6], 
        u_diff = 0.5, 
    )
    
    exp2 = run_model_opt(
        cost = x[0], 
        A = x[1], 
        B = x[2], 
        u_skill = x[3], 
        tao_skill = x[4], 
        skill = x[5], 
        scale = x[6], 
        u_diff = x[7], 
    )
    
    exp3 = run_model_opt(
        cost = x[8], 
        A = x[1], 
        B = x[2], 
        u_skill = x[3], 
        tao_skill = x[4], 
        skill = x[5], 
        scale = x[6], 
        u_diff = 0.5, 
    )        

In [ ]:
h=list(human_means[0])+list(human_means[1])+list(human_means[2])

In [ ]:
m=list(exp1)+list(exp2)+list(exp3)


In [ ]:
zip(h,m)

### not fitting correctly
* baseline hard
* moral low effort success and baseline

In [ ]:
plt.plot(m,h,'.')
np.corrcoef(m,h)